# Setup

In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# unzip data
!unzip -q "/images.zip" -d images

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

c:\Users\Burhani\Documents\Lacuna Solar Survey Challenge--Zindi\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Imports

In [4]:
# Read in the training dataset
train = pd.read_csv(f"Train.csv")

In [5]:
train.head()

,ID,img_origin,placement,boil_nbr,pan_nbr,polygon
0,ID00rw8,D,roof,0,2,"[(2087, 2179.0), (2181, 2191.0), (2171, 2223.0..."
1,ID014O6EC7,D,roof,0,1,"[(1327, 1574.0), (1595, 1308.0), (2169, 1744.0..."
2,ID020cu0z,D,openspace,0,1,"[(2215, 1372.0), (2400, 1422.0), (2316, 1716.0..."
3,ID024YTBkLvRpQahT,D,roof,0,1,"[(1574, 526.0), (1698, 366.0), (1810, 422.0), ..."
4,ID024YTBkLvRpQahT,D,roof,0,2,"[(1402, 782.0), (1814, 854.0), (1726, 1074.0),..."


In [6]:
# Create a placement mapper
placement_mapper = train[["ID", "placement"]].drop_duplicates().set_index("ID").to_dict()
# Create a "img_origin" mapper
img_origin_mapper = train[["ID", "img_origin"]].drop_duplicates().set_index("ID").to_dict()

# Group by "ID" and sum up boil_nb, pan_nbr
train_df = train.groupby("ID").sum().reset_index()[["ID", "boil_nbr", "pan_nbr"]]

# Map img_origin and placement
train_df["img_origin"] = train_df["ID"].map(img_origin_mapper["img_origin"])
train_df["placement"] = train_df["ID"].map(placement_mapper["placement"])

# Create path column
train_df["path"] = "images/" + train_df["ID"] + ".jpg"

In [7]:
train_df.head()

,ID,boil_nbr,pan_nbr,img_origin,placement,path
0,ID00rw8,0,2,D,roof,images/ID00rw8.jpg
1,ID014O6EC7,0,1,D,roof,images/ID014O6EC7.jpg
2,ID020cu0z,0,1,D,openspace,images/ID020cu0z.jpg
3,ID024YTBkLvRpQahT,0,5,D,roof,images/ID024YTBkLvRpQahT.jpg
4,ID02vByTw8Htl,0,1,D,roof,images/ID02vByTw8Htl.jpg


In [8]:
'''# Stratified KFold based on multi-label targets
train_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
train_df["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
    train_df.loc[valid_idx, "fold"] = fold'''

'# Stratified KFold based on multi-label targets\ntrain_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)\nskf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)\ntrain_df["fold"] = -1\nfor fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):\n    train_df.loc[valid_idx, "fold"] = fold'

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_df["fold"] = -1

for fold, (_, valid_idx) in enumerate(kf.split(train_df)):
    train_df.loc[valid_idx, "fold"] = fold

In [10]:
# Define Transformations
train_transforms = A.Compose([
    A.Resize(384, 384),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(384, 384),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [11]:
# Custom Dataset
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = cv2.imread(row["path"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)["image"]

        if self.to_train:
          target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
          return image, target
        else:
          return image

In [12]:
# Prepare Dataloaders
fold = 0  # Change fold index as needed
train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)

dataset_train = SolarPanelDataset(train_data, transform=train_transforms)
dataset_valid = SolarPanelDataset(valid_data, transform=test_transforms)

train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True, num_workers=os.cpu_count())
valid_loader = DataLoader(dataset_valid, batch_size=32, shuffle=False)

In [13]:
# Model Definition
class EfficientNetRegressor(nn.Module):
    def __init__(self):
        super(EfficientNetRegressor, self).__init__()
        self.model = timm.create_model("tf_efficientnet_b1", pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 2)

    def forward(self, x):
        return self.model(x)

In [14]:
# Training Setup
model = EfficientNetRegressor().to(torch.device("cpu"))
criterion = nn.L1Loss()  # MAE Loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Model Saving Path
best_model_path = "best_model.pth"

In [ ]:
# Training Loop
num_epochs = 3
best_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, targets = images.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, targets = images.cuda(), targets.cuda()
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

    # Save Best Model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 1/3 - Training:   0%|          | 0/83 [00:00<?, ?it/s]

In [ ]:
# Load Best Model
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Predict on Validation Set
preds = []
true_vals = []
with torch.no_grad():
    for images, targets in tqdm(valid_loader, desc="Predicting on Validation Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        preds.append(outputs)
        true_vals.append(targets.numpy())
preds = np.concatenate(preds, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

<ipython-input-24-ce980cad07c1>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
Predicting on Validation Set: 100%|██████

In [ ]:
from sklearn.metrics import mean_absolute_error

# Evaluate using MAE
mae = mean_absolute_error(true_vals, preds)
print(f"Validation MAE: {mae:.4f}")

Validation MAE: 2.1703


In [ ]:
# Predict on Test Set
test_df = pd.read_csv(f"Test.csv")

test_df["path"] = "images/" + test_df["ID"] + ".jpg"

dataset_test = SolarPanelDataset(test_df, transform=test_transforms, to_train=False)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

test_preds = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.cuda()
        outputs = model(images).cpu().numpy()
        test_preds.append(outputs)
test_preds = np.concatenate(test_preds, axis=0)

Predicting on Test Set: 100%|██████████| 35/35 [03:06<00:00,  5.33s/it]


In [ ]:
# Create Sample Submission
submission = pd.DataFrame()
submission["ID"] = np.repeat(test_df["ID"].values, 2)
submission["ID"] = submission["ID"] + np.tile(["_boil", "_pan"], len(test_df))
submission["Target"] = test_preds.flatten().clip(0,1000)

# Save Submission
submission.to_csv("SampleSubmission.csv", index=False)
print("Sample submission saved!")

Sample submission saved!
